In [91]:
import sys
sys.path.append('..')
import doctable

import typing
import datetime

In [92]:
class UserID(str):
    pass

@doctable.table_schema(
    table_name='user_records', 
    slots = True,
    indices={
        'user_ind_rqts_uid': doctable.Index('request_ts', 'uid', unique=True),
    },
)
class UserRecord:
    request_ts: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(nullable=False, index=True),
    )
    uid: UserID = doctable.Column(
        column_args=doctable.ColumnArgs(nullable=False, index=True),
    )
    stats: doctable.JSON = doctable.Column()
    
    id: int = doctable.Column(
        column_args=doctable.ColumnArgs(
            order = 0,
            primary_key=True,
            autoincrement=True,
        ),
    )
    added: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(
            default=datetime.datetime.now, 
            onupdate=datetime.datetime.now
        ),
        field_args = doctable.FieldArgs(
            repr=False, # don't show this field when printing
        )
    )
    
    @classmethod
    def from_api(cls, 
        request_ts: datetime.datetime,
        uid: str,
        stats: typing.Dict[str, typing.Any] = None,
    ) -> 'UserRecord':
        return cls(
            request_ts=request_ts,
            uid=uid,
            stats=stats if stats is not None else {},
        )
doctable.inspect_schema(UserRecord).column_info_df()

,Col Name,Col Type,Attr Name,Hint,Order,Primary Key,Foreign Key,Index,Default
0,id,Integer,id,int,"(0, 3)",True,False,None,None
1,request_ts,DateTime,request_ts,datetime,"(inf, 0)",False,False,True,None
2,uid,String,uid,str,"(inf, 1)",False,False,True,None
3,stats,JSON,stats,JSON,"(inf, 2)",False,False,None,None
4,added,DateTime,added,datetime,"(inf, 4)",False,False,None,now


In [93]:
UserRecord.from_api(
    request_ts = datetime.datetime.utcnow(),
    uid = '5678',
)

UserRecord(request_ts=datetime.datetime(2023, 12, 4, 17, 59, 2, 357393), uid='5678', stats={}, id=MISSING)

In [94]:
class PostID(str):
    pass

@doctable.table_schema(
    table_name='post_records', 
    slots = True,
    indices={
        'post_ind_rqts_pid': doctable.Index('request_ts', 'pid', unique=True),
    },
    constraints=[
        doctable.ForeignKey(
            local_columns = ['user_request_id', 'uid'], 
            from_columns=['user_records.request_ts', 'user_records.uid'],
        ),
    ]
)
class PostRecord:
    request_ts: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(nullable=False, index=True),
    )
    pid: PostID = doctable.Column(
        column_args=doctable.ColumnArgs(nullable=False, index=True),
    )
    uid: UserID = doctable.Column(
        column_args=doctable.ColumnArgs(nullable=False),
    )
    user_request_id: datetime.datetime = doctable.Column()
    
    parent_pid: str = doctable.Column()
    engagement: doctable.JSON = doctable.Column()
    
    id: int = doctable.Column(
        column_args=doctable.ColumnArgs(
            order = 0,
            primary_key=True,
            autoincrement=True,
        ),
    )
    added: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(
            default=datetime.datetime.now, 
            onupdate=datetime.datetime.now
        ),
        field_args = doctable.FieldArgs(
            repr=False, # don't show this field when printing
        )
    )
    
    @classmethod
    def from_api(cls, 
        request_ts: datetime.datetime,
        pid: str,
        uid: str,
        parent_pid: str = None,
        engagement: typing.Dict[str, typing.Any] = None,
    ) -> 'PostRecord':
        return cls(
            request_ts=request_ts,
            pid=pid,
            uid=uid,
            parent_pid=parent_pid,
            engagement=engagement if engagement is not None else {},
        )
doctable.inspect_schema(PostRecord).column_info_df()

,Col Name,Col Type,Attr Name,Hint,Order,Primary Key,Foreign Key,Index,Default
0,id,Integer,id,int,"(0, 6)",True,False,None,None
1,request_ts,DateTime,request_ts,datetime,"(inf, 0)",False,False,True,None
2,pid,String,pid,str,"(inf, 1)",False,False,True,None
3,uid,String,uid,str,"(inf, 2)",False,False,None,None
4,user_request_id,DateTime,user_request_id,datetime,"(inf, 3)",False,False,None,None
5,parent_pid,String,parent_pid,str,"(inf, 4)",False,False,None,None
6,engagement,JSON,engagement,JSON,"(inf, 5)",False,False,None,None
7,added,DateTime,added,datetime,"(inf, 7)",False,False,None,now


In [95]:
PostRecord.from_api(
    request_ts = datetime.datetime.utcnow(),
    pid = '1234',
    uid = '5678',
)

PostRecord(request_ts=datetime.datetime(2023, 12, 4, 17, 59, 2, 474697), pid='1234', uid='5678', user_request_id=MISSING, parent_pid=None, engagement={}, id=MISSING)

In [96]:

import dataclasses
import collections

null_user_record = UserRecord.from_api(
    request_ts=datetime.datetime.utcfromtimestamp(0), 
    uid='NULL',
)

@dataclasses.dataclass
class SMDB:
    core: doctable.ConnectCore
    user_table: doctable.DBTable
    post_table: doctable.DBTable
    
    @classmethod
    def open(cls, target: str) -> 'SMDB':
        core = doctable.ConnectCore.open(target=target, dialect='sqlite')
        with core.begin_ddl() as ddl:
            core.enable_foreign_keys()
            ut = ddl.create_table_if_not_exists(UserRecord)
            pt = ddl.create_table_if_not_exists(PostRecord)
            
        with ut.query() as q:
            q.insert_single(null_user_record, ifnotunique='ignore')
        
        return cls(core=core, user_table=ut, post_table=pt)
    
    def select_user_requests(self, **select_kwargs) -> typing.Dict[UserID, typing.List[datetime.datetime]]:
        '''Get a list of request timestamps grouped by user.'''
        with self.core.query() as q:
            cols = self.user_table.cols('request_ts', 'uid')
            udata = q.select(cols, order_by=self.user_table['request_ts'], **select_kwargs).all()
        
        user_requests: typing.Dict[UserID, typing.List[datetime.datetime]] = collections.defaultdict()
        for rqts, uid in udata:
            user_requests[uid].append(rqts)
            
        return user_requests
    
    def insert_users(self, users: typing.List[UserRecord], **insert_kwargs):
        with self.user_table.query() as q:
            q.insert_multi(users, **insert_kwargs)
            
    def insert_posts(self, posts: typing.List[PostRecord], **insert_kwargs):
        with self.post_table.query() as q:
            q.insert_multi(posts, **insert_kwargs)

In [97]:

with ut.query() as q:
    q.insert_single(null_user_record, ifnotunique='replace')
    
with core.query() as q:
    print(q.select(ut.all_cols()).df())

   id request_ts   uid stats                      added
0   1 1970-01-01  NULL    {} 2023-12-04 12:59:02.596908


In [100]:
with pt.query() as q:
    q.insert_single(PostRecord.from_api(
        request_ts=datetime.datetime.utcnow(), 
        pid='1234', 
        uid='NULL',
    ))

In [99]:
with core.query() as q:
    print(q.select(pt.all_cols()).df())

   id                 request_ts   pid   uid user_request_id parent_pid  \
0   1 2023-12-04 17:59:02.647134  1234  NULL            None       None   

  engagement                      added  
0         {} 2023-12-04 12:59:02.650424  
